In [1]:
# General libraries
import pandas as pd
import numpy as np
import datetime as dt
import time

# Scrape
from bs4 import BeautifulSoup #To use containers
from pprint import pprint #To print pretty
import re
import urllib
import urllib.request
import urllib.parse
import urllib.error
import ssl
import json
import ast
import os
from urllib.request import Request, urlopen

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [3]:
from selenium import webdriver 
import pandas as pd 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [25]:
ROOT_URL = "https://www.youtube.com" #Define mother URL
ADD_CHANNEL = '/channel/UCDiNP9EvWRTPUyjdm20p_fA/videos'

# We need the headers because otherwise we get blocked

headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:66.0) Gecko/20100101 Firefox/66.0"}

In [26]:
def getListingLinks(link):
    # Open the driver
    driver =  webdriver.Chrome("C:/Users/Álvaro/Downloads/chromedriver.exe")
    driver.maximize_window()
    driver.get(link)
    
    time.sleep(1)

    elem = driver.find_element_by_tag_name("body")

    no_of_pagedowns = 1

    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.2)
        no_of_pagedowns-=1

    # Save the links
    listing_links = []
    links = driver.find_elements_by_xpath('//*[@id=\"video-title\"]')
    for link in links:
        listing_links.append(str(link.get_attribute('href')))
    driver.close()
    return listing_links

In [27]:
links = getListingLinks(ROOT_URL+ADD_CHANNEL)

In [28]:
raw_data = pd.DataFrame(columns = ['Name','Likes','Unlikes','Views','Pub_date'])
raw_data.head() 

,Name,Likes,Unlikes,Views,Pub_date


In [31]:

i = 0
kpis = []

for each_link in links[:1]: 
    # For ignoring SSL certificate errors

    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    # Input from user
    url = each_link

    # Making the website believe that you are accessing it using a mozilla browser

    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    # Creating a BeautifulSoup object of the html page for easy extraction of data.

    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')
    video_details = {}
    

    for span in soup.findAll('span',attrs={'class': "title style-scope ytd-video-primary-info-renderer"}):
        video_details['TITLE'] = span.text.strip()

    for meta in soup.findAll('meta',attrs={'itemprop': 'datePublished'}):
        video_details['PUBLICATION_DATE'] = meta['content']

    for div in soup.findAll('div',attrs={'class': 'watch-view-count'}):
        video_details['NUMBER_OF_VIEWS'] = div.text.strip()

    for button in soup.findAll('button',attrs={'class': 'style-scope ytd-toggle-button-renderer style-default-active'}):
        video_details['LIKES'] = button.text.strip()

    for button in soup.findAll('button',attrs={'class': 'yt-uix-button yt-uix-button-size-default yt-uix-button-opacity yt-uix-button-has-icon no-icon-markup like-button-renderer-dislike-button like-button-renderer-dislike-button-unclicked yt-uix-clickcard-target yt-uix-tooltip'}):
        video_details['DISLIKES'] = button.text.strip()
        
        
    try:
    # block raising an exception
        kpis.append([video_details['TITLE'],video_details['LIKES'],video_details['DISLIKES'],video_details['NUMBER_OF_VIEWS'],video_details['PUBLICATION_DATE']])
    except:
        pass # doing nothing on exception    

    i = i+1
  
    print (f'''---------- Video {i} extraction of data is complete----------''')
    
raw_data = pd.DataFrame(kpis, columns = ['Name','Likes','Dislikes','Views','Pub_date'] )
raw_data.head()

---------- Video 1 extraction of data is complete----------


,Name,Likes,Dislikes,Views,Pub_date


In [32]:
raw_data

,Name,Likes,Dislikes,Views,Pub_date
